# Seminar 1. Intro to Pyro

## **Pyro** - вероятностное программирование с pytorch


Нам нужен будет пакет pyro.

In [ ]:
!pip install pyro-ppl

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch.distributions import constraints

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

pyro.set_rng_seed(101)

## **Распределение в pytorch**

Распределения в Pyro - wrapper для распределений в pytorch, потому что нам хочется дифференцировать автоматически и работать с тензорами, причем делать это быстро с помощью возможностей pytorch.

In [ ]:
loc = 0.   # mean zero
scale = 1. # unit standard deviation

In [ ]:
# create a normal distribution object in torch
normal = torch.distributions.Normal(loc, scale) 



Распределение - объект, у которого есть два метода: sample() и log_prob().

* Первый возвращает объект из этого распределения. 
* Второй возвращает логарифм плотности для наблюдения.

In [ ]:
# with rsample it is possible to propagate through random variables 
# in a computational graph
x = normal.rsample() # draw a sample from N(0, 1) 

print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0, 1)

## **Reparametrization trick**

Когда мы сэмплируем объекты из torch.distribution, мы можем считать градиенты по параметрам

In [ ]:
loc = torch.tensor(0., requires_grad=True)  # mean zero
scale = 2 
normal = torch.distributions.Normal(loc, scale) 

In [ ]:
x = normal.rsample() # draw a sample from N(0, 2) 
x.backward()
print(loc.grad)

In [ ]:
loc = torch.tensor(1., requires_grad=True)  # mean zero
scale = torch.tensor(2., requires_grad=True)  # mean zero
normal = torch.distributions.Normal(loc, scale) 

In [ ]:
x = normal.rsample() # draw a sample from N(0, 1) 
x.backward()
print("x = {:.4}".format(x.item()))
print("loc derivative = {:.4}, scale derivative = {:.4}".format(loc.grad.item(), 
                                                                scale.grad.item()))

# zero grads
loc.grad.data.zero_();
scale.grad.data.zero_();

## **Распределение в pyro**

Обратите внимание, что теперь у каждого распределения есть название. Бэкэнд Pyro использует эти имена для уникальной идентификации состояния и изменения сэмплов.

In [ ]:
loc = torch.tensor(1., requires_grad=True)  # mean zero
scale = torch.tensor(2., requires_grad=True)  # mean zero

x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
x.backward()
print("x = {:.4}".format(x.item()))
print("loc derivative = {:.4}, scale derivative = {:.4}".format(loc.grad.item(),                                                          scale.grad.item()))

**Вопрос**: Что было бы, если бы мы не переопределили loc и scale заново?

## **Сложное распределение в pyro**

In [ ]:
def weather(p):
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(p))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 15.0, 'sunny': 25.0}[cloudy]
    scale_temp = {'cloudy': 15.0, 'sunny': 10.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp

for _ in range(3):
    print(weather(0.5))

In [ ]:
x = np.array([[x[0]=='sunny', x[1].item()] for _ in range(5000) for x in [weather(0.3)]])
sns.histplot(x[:,1][x[:,0]==0], stat='density')
sns.histplot(x[:,1][x[:,0]==1], stat='density', color='tab:orange')
plt.show()

Можно задавать распределения динамически, пока имена у них уникальные.

В качестве примера рассмотрим геометрическое распределение: Количество неудач до первого успеха.

In [ ]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), 
                    pyro.distributions.Bernoulli(p))
    if x.item() == 1: # Success
        return 0
    else:
        return 1 + geometric(p, t + 1)

In [ ]:
sns.histplot([geometric(0.5) for _ in range(1000)])
plt.show()

## ------
## **Упражнение:**

Написать реализацию случайного блуждания на pyro: 

* начинаем с 0 в нулевой момент времени
* с вероятностью p идем вверх, с вероятностью (1-p) --- вниз
* return -- момент времени, когда абсолютное значение больше 10.

## ------

Можно строить граф распределений. 

Например построим произведение двух независмых величин из распределения N(loc, scale), где loc ~ N(0,1), а scale мы задаем сами. Градиент при этом передается через параметры распределения.

In [ ]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

scale = torch.tensor(1.0, requires_grad=True)
print(make_normal_normal()(scale))

## ------
## **Упражнение:**

Написать реализацию распределения Стьюдента (по определению) на pyro:

Определение: распределение Стьюдента c n степенями свободы - t(n):

$$
Y_0, Y_1, ..., Y_n - \text{i.i.d}\; \mathcal{N}(0,1)\\
T = \frac{Y_0}{\sqrt{\frac{1}{n}\sum_{i=1}^{n}Y_i^2}} \sim t(n)
$$

In [ ]:
def student(n):
    """
    * n - количество степеней свободы
    """
    pass

Сравним вашу реализацию с реалищацией из numpy

In [ ]:
import scipy.stats as stats

my_student = [student(10).item() for _ in range(1000)]
np_student = np.random.standard_t(10, 1000)
x = np.percentile(my_student, np.linspace(1, 99, num=100))
y = np.percentile(np_student, np.linspace(1, 99, num=100))
plt.scatter(x, y)

## ------

# **Зачем нам все это было нужно?**

-Нам хочется работать со сложными распределениями, например семплировать из условных распределений.

## **Пример**

Рассмотрим более сложный пример. Рассмотрим весы с шумом (noisy scale). Не путайте scale (масштаб) в нормальном распределение и scale (весы) в нашем примере. 

In [ ]:
def scale(guess):
    """
    Мы определеяем один параметр: 
    * guess - априорное предположение о весе 

    Далее задаем вероятностную модель:
    * weight - какой вес получился
    * measurement - как мы его измерили
    """
    
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    return pyro.sample("measurement", dist.Normal(weight, 0.75))

**Вопрос**: Какая вероятностная модель получилась?

$$
\text{weight | guess} ∼ \mathcal{N}(\text{guess}, 1)
$$

$$
\text{measurement | guess, weight} ∼ \mathcal{N}(\text{weight}, 0.75)
$$

## **Получение условных вероятностей**

Теперь мы начинаем использовать имена наших переменных.

Укажем, что мы уже знаем значение weight равное 7. И посмотрим какое итоговое распределение мы получим.

In [ ]:
conditioned_scale = pyro.condition(scale, data={"weight": 7})

guess = 2
print(f"guess: {guess}, measurement: {conditioned_scale(guess).item()}")

guess = 20
print(f"guess: {guess}, measurement: {conditioned_scale(guess).item()}")

**Вопрос**: Почему нет влияния априорного значения веса guess?

Рассмотрим теперь более интересный случай. Где мы знаем значение итогого измерения measurement. И хотим найти апостериорное распределение веса weight.

In [ ]:
measurement_value = torch.tensor(9.5) # Tensor
conditioned_scale = pyro.condition(scale, data={"measurement": measurement_value})

Далее нам надо определить как будет искаться апостериорное распределение.

Для этого используется функция которую принято называть **guide**.

В нашем примере параметры апостериорного распределения могут быть посчитаны аналитически.

In [ ]:
def get_conditional_location_scale(guess, measurement, measurement_variance):
    loc = ((measurement_variance**2 * guess + measurement) / 
           (1 + measurement_variance**2)) # 9.14
    scale = np.sqrt(measurement_variance**2/ (1 + measurement_variance**2)) # 0.6
    return loc, scale

Например для конкретного значения guess:

In [ ]:
guess = 8.5
loc, w_scale = get_conditional_location_scale(guess, 9.5, 0.75)
loc, w_scale

А значит можно **guide** можно написать явно:

In [ ]:
def perfect_guide(guess, measurement=9.5, measurement_variance=0.75):
    loc, scale = get_conditional_location_scale(guess, measurement, measurement_variance)
    return pyro.sample("weight", dist.Normal(loc, scale))

## **Байесовский вывод**

Предположим, что мы не можем выразить апостериорное распределение аналитически.

Попробуем оценить апостериорное распределение с помощью pyro и вычислительных методов.
Для этого зададим параметрическое семейство в качестве апостериорного и будем оценивать параметры этого распределения.

В качестве параметрического семейства выберем нормальное распределение N(a,b).

In [ ]:
def scale_parametrized_guide(guess):
    a = pyro.param("a", torch.tensor(guess))
    b = pyro.param("b", torch.tensor(1.)) 
    return pyro.sample("weight", dist.Normal(a, torch.abs(b)))

Так как стандартное отклонение должно быть положительным, мы использовали torch.abs. Но можно воспользоваться и встроенным набором ограничений на параметры:

In [ ]:
def scale_parametrized_guide(guess):
    a = pyro.param("a", torch.tensor(guess))
    b = pyro.param("b", torch.tensor(1.), constraint=constraints.positive) # <--
    return pyro.sample("weight", dist.Normal(a, b))  # no more torch.abs

Реализуем теперь поиск апостериорных параметров с помощью стохастического вариационного вывода (SVI). Более детально мы с ним ознакомимся позднее.

In [ ]:
guess = 8.5

pyro.clear_param_store()
svi = pyro.infer.SVI(model=conditioned_scale,
                     guide=scale_parametrized_guide,
                     optim=pyro.optim.SGD({"lr": 0.001, "momentum": 0.1}),
                     loss=pyro.infer.Trace_ELBO())

In [ ]:
losses, a, b  = [], [], []
num_steps = 2500
for t in range(num_steps):
    losses.append(svi.step(guess))
    a.append(pyro.param("a").item())
    b.append(pyro.param("b").item())

In [ ]:
plt.plot(losses)
plt.title("ELBO")
plt.xlabel("step")
plt.ylabel("loss");
print('a = ', pyro.param("a").item())
print('b = ', pyro.param("b").item())

In [ ]:
loc, w_scale = get_conditional_location_scale(guess, 9.5, 0.75)

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.xlabel('step')
plt.ylabel('a')
plt.axhline(loc, ls=':', c='black')
plt.plot(a)


plt.subplot(1, 2, 2)
plt.xlabel('step')
plt.ylabel('b')
plt.axhline(w_scale, ls=':', c='black')
plt.plot(b)

plt.show()

Как мы видим, результаты получились очень близкие.

## **Работа с неаналитическим распределением**

## ------
## **Упражнение:**

Написать процедуру Байесовского вывода для более сложной, аналитически не вычислимой модели.

Рассмотрим предыдущую задачу, но изменим следующее распределение:

$$
\text{weight | guess} ∼ \mathcal{N}(\text{guess}, 1)
$$

$$
\text{measurement | guess, weight} ∼ \mathcal{N}(\color{red}{f(}\text{weight}\color{red}{)}, 0.75)
$$

При этом $f(\cdot)$ является нелинейной функцией.

Пусть $f(x) = \sqrt x$<br>
Найдите параметры апостериорного распределения weight при заданных<br>
guess = 8.5<br>
measurement = 4.0<br>

## ------

### EOF